In [1]:
import geojson
import os
import pandas as pd
import cv2
import numpy as np
from sklearn.metrics import cohen_kappa_score
from glob import glob

In [16]:
def json_load(json_dir,geofile1):
    with open(os.path.join(json_dir,geofile1)) as f:
        gj = geojson.load(f)
    features = gj['features']
    return features


def polygon_rater_data_load(json_dir,geofile1):
    features1 = json_load(json_dir,geofile1)
    feature1_points = [f for f in features1 if (f["geometry"]["type"]=="Polygon") & ("isLocked" not in f["properties"].keys())]
    feature1_coords = [cords["geometry"]["coordinates"] for cords in feature1_points]
    feature1_name = []
    feature1_class = []
    for cords in feature1_points:
        if "name" not in cords["properties"].keys():
            feature1_name.append(None)
        else:
            feature1_name.append(cords["properties"]["name"])
        if "classification" not in cords["properties"].keys():
            feature1_class.append(None)
        elif "name" not in cords["properties"]["classification"].keys():
            feature1_class.append(None)
        else:
            feature1_class.append(cords["properties"]["classification"]["name"])
    feature1_df = pd.DataFrame({"coordinates":feature1_coords,"class":feature1_class, "name":feature1_name})
    return feature1_coords, feature1_class, feature1_name, feature1_df


def point_rater_data_load(json_dir,geofile2):
    features2 = json_load(json_dir,geofile2)
    features2_points = [f for f in features2 if f["geometry"]["type"]=="Point"]
    feature2_coords = [cords["geometry"]["coordinates"] for cords in features2_points]
    feature2_class = [cords["properties"]["classification"]["name"] if "classification" in (cords["properties"].keys()) else "None" for cords in features2_points ]
    feature2_name = [cords["properties"]["name"] if "name" in (cords["properties"].keys()) else "None" for cords in features2_points]
    feature2_df = pd.DataFrame({"coordinates":feature2_coords,"class":feature2_class, "name":feature2_name})
    return feature2_coords, feature2_class, feature2_name, feature2_df

def match_main_point_rater(feature1_coords,feature1_class,feature1_name,feature2_coords,feature2_class,feature2_name, radius):
    match = []
    i=0
    for c, class_var, name_var in zip(feature1_coords,feature1_class,feature1_name):
        #print(contours,class_var,name_var)
        j=0
        for cnt, class_var1, name1 in zip(feature2_coords,feature2_class,feature2_name):
            #print(c, class_var1, name1)
            #print((int(c[0]),int(c[1])))
            
            contours = [[int(c[0])+radius,int(c[1])],[int(c[0])+0.5*radius,int(c[1])+0.5*radius], [int(c[0]),int(c[1])+radius], [int(c[0])-0.5*radius,int(c[1])+0.5*radius], [int(c[0])-radius,int(c[1])], 
                        [int(c[0])-0.5*radius,int(c[1])-0.5*radius], [int(c[0])-radius,int(c[1])],[int(c[0])+0.5*radius,int(c[1])-0.5*radius]]
            #print(len(cnt[0]))
            if (len(cnt[0])>2):
                cnt = np.mean(cnt[0], axis=0)
            #print(len(cnt))
            dist = cv2.pointPolygonTest(np.int32(np.array(contours).round()),(int(cnt[0]),int(cnt[1])),False)
            if dist>=1:
                match.append([i,class_var,name_var,j,class_var1,name1, contours, cnt])
            j=j+1
        i=i+1 
    df = pd.DataFrame(match, columns=["main_rater_index","main_rater_class","main_rater_object_name","rater1_index","rater1_class","rater1_object_name","polygon_coords","point_coords"])
    return df



def match_point_rater(feature1_coords,feature1_class,feature1_name,feature2_coords,feature2_class,feature2_name):
    match = []
    i=0
    for contours, class_var, name_var in zip(feature1_coords,feature1_class,feature1_name):
        #print(contours,class_var,name_var)
        j=0
        for c, class_var1, name1 in zip(feature2_coords,feature2_class,feature2_name):
            #print(c, class_var1, name1)
            #print((int(c[0]),int(c[1])))
            dist = cv2.pointPolygonTest(np.int32(np.array(contours).round()),(int(c[0]),int(c[1])),False)
            if dist>=1:
                match.append([i,class_var,name_var,j,class_var1,name1, contours, c])
            j=j+1
        i=i+1 
    df = pd.DataFrame(match, columns=["main_rater_index","main_rater_class","main_rater_object_name","rater1_index","rater1_class","rater1_object_name","polygon_coords","point_coords"])
    return df




def compute_kappa_score(df, rater1_column, rater2_column):
    #df1= df[~((df["main_rater_class"]=='None') & (df["main_rater_class"]=='None'))]
    df1=df
    print(len(df1))
    labeler1 = df1[rater1_column]
    labeler2 = df1[rater2_column]
    return cohen_kappa_score(labeler1, labeler2)

def find_missing_main_rater(df, feature1_coords,feature1_class,feature1_name):
    missing_main_rater_list = []
    for i in range(len(feature1_coords)):
        if i not in df["main_rater_index"].values:
            print(i,feature1_name[i], feature1_coords[i])
            missing_main_rater_list.append([i,feature1_class[i],feature1_name[i],None, None,None, feature1_coords[i],None])
    missing_main_rater = pd.DataFrame(missing_main_rater_list, columns=["main_rater_index","main_rater_class","main_rater_object_name","rater1_index","rater1_class","rater1_object_name","polygon_coords","point_coords"])
    return missing_main_rater

def find_missing_rater1(df, feature2_coords,feature2_class,feature2_name):
    missing_rater1_list = []
    for i in range(len(feature2_class)):
        if i not in df["rater1_index"].values:
            print(i,feature2_class[i],feature2_coords[i] )
            missing_rater1_list.append([None, None,None,i,feature2_class[i],feature2_name[i], None, feature2_coords[i]])
    missing_rater1 = pd.DataFrame(missing_rater1_list,columns=["main_rater_index","main_rater_class","main_rater_object_name","rater1_index","rater1_class","rater1_object_name","polygon_coords","point_coords"])
    return missing_rater1

def map_class(l):
    if l=="Cored":
        return "(C)"
    if l=="Diffuse":
        return "(D)"
    if l=="Mature":
        return "(M)"
    if l=="Pre":
        return "(P)"
    if l=="Ghost":
        return "(G)"
    if l=="Coarse-Grained":
        return "(CG)"
    return "None"


def find_match(json_dir1,json_dir2, geojsons_names,radius):
    all_geojson_df = pd.DataFrame()
    for geofile1 in geojsons_names:
        print("------------------",geofile1,"---------------------")
        #feature1_coords, feature1_class, feature1_name, feature1_df = point_rater_data_load(json_dir1,geofile1)
        feature1_coords, feature1_class, feature1_name, feature1_df = polygon_rater_data_load(json_dir1,geofile1)
        print(len(feature1_coords))
        feature2_coords, feature2_class, feature2_name, feature2_df = point_rater_data_load(json_dir2,geofile1)
        #feature2_coords, feature2_class, feature2_name, feature2_df = polygon_rater_data_load(json_dir2,geofile1)
        print(len(feature2_coords))
        #df = match_main_point_rater(feature1_coords,feature1_class,feature1_name,feature2_coords,feature2_class,feature2_name, radius)
        df = match_point_rater(feature1_coords,feature1_class,feature1_name,feature2_coords,feature2_class,feature2_name)
        missing_main_rater = find_missing_main_rater(df, feature1_coords,feature1_class,feature1_name)
        #missing_rater1 = find_missing_rater1(df, feature2_coords,feature2_class,feature2_name)
        #df_final = pd.concat([df,missing_rater1,missing_main_rater], axis=0, ignore_index=True) 
        df_final = pd.concat([df,missing_main_rater], axis=0, ignore_index=True) 
        df_final["main_rater_class"] = np.where(df_final["main_rater_class"].isna(),"None",df_final["main_rater_class"])
        df_final["rater1_class"] = np.where(df_final["rater1_class"].isna(),"None",df_final["rater1_class"])
        df_final["main_rater_annotation"] = df_final["main_rater_class"].apply(lambda l: "Polygon"+ map_class(l) if l!='None' else "")
        df_final["rater1_annotation"] = df_final["rater1_class"].apply(lambda l: "Point"+ map_class(l) if l!='None' else "")
        df_final["geojson_file"] = geofile1
        if len(all_geojson_df)==0:
            all_geojson_df = df_final
        else:
            all_geojson_df =  pd.concat([all_geojson_df,df_final], ignore_index=True)
    return all_geojson_df
        

In [17]:
#json_dir2= "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/interrater-study/interrater_geojson_Max/Brittany"
json_dir2= "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/interrater-study/interrater_geojson_Max/Harry"
json_dir1= "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/interrater-study/interrater_geojson_Max/Max"

In [18]:
geojsons_main_rater =  glob(os.path.join(json_dir1,"*.geojson"))
geojsons_names =  [x.split("/")[-1] for x in geojsons_main_rater]

In [19]:
geojsons_names

['22650_7_AmyB_1_ceren.mrxs.geojson',
 '25144_1_AmyB_1_ceren.mrxs.geojson',
 '22640_1_AmyB_1_ceren.mrxs.geojson',
 '30414_6_AmyB_1_ceren.mrxs.geojson',
 '420418_6_AmyB_1_ceren.mrxs.geojson',
 '354049_6_AmyB_1_ceren.mrxs.geojson']

In [20]:
max_output = find_match(json_dir1,json_dir2, geojsons_names,250)

------------------ 22650_7_AmyB_1_ceren.mrxs.geojson ---------------------
15
25
------------------ 25144_1_AmyB_1_ceren.mrxs.geojson ---------------------
29
20
13 0:Cored [[[25055, 95404], [25051, 95408], [25032, 95408], [25028, 95411], [25017, 95411], [25014, 95413], [25006, 95413], [25003, 95417], [24997, 95417], [24996, 95418], [24987, 95418], [24969, 95438], [24965, 95436], [24956, 95436], [24951, 95438], [24943, 95438], [24942, 95440], [24936, 95440], [24933, 95444], [24931, 95445], [24927, 95449], [24925, 95449], [24922, 95451], [24918, 95454], [24913, 95454], [24913, 95458], [24911, 95460], [24907, 95460], [24904, 95465], [24902, 95465], [24898, 95469], [24893, 95472], [24889, 95474], [24887, 95478], [24884, 95480], [24878, 95483], [24878, 95487], [24875, 95489], [24873, 95492], [24868, 95498], [24864, 95499], [24864, 95503], [24862, 95508], [24859, 95512], [24859, 95514], [24857, 95517], [24857, 95519], [24853, 95523], [24853, 95525], [24853, 95532], [24850, 95534], [24850, 9

In [21]:
len(max_output[max_output["rater1_class"]=="None"])

26

In [22]:
def find_equiv_diameter(cnt):
    area = cv2.contourArea(cnt)
    equi_diameter = np.sqrt(4*area/np.pi)
    return equi_diameter

In [23]:
max_output["area"] = max_output["polygon_coords"].apply(lambda l: cv2.contourArea(np.int32(np.array(l).round())))

In [24]:
max_output["diameter"] = max_output["area"].apply(lambda l: np.sqrt(4*l/np.pi))

In [25]:
max_output["size_bracket"] = np.where(max_output["diameter"]<=10,"a. <=10",np.where(max_output["diameter"]<=50,"b. 10-50", 
                                                                                   np.where(max_output["diameter"]<=100,"c. 50-100",
                                                                                            np.where(max_output["diameter"]<=250,"d. 100-250",
                                                                                                     np.where(max_output["diameter"]<=500,"e. 250-500",
                                                                                                     "f. >500")))))

In [26]:
max_output.columns

Index(['main_rater_index', 'main_rater_class', 'main_rater_object_name',
       'rater1_index', 'rater1_class', 'rater1_object_name', 'polygon_coords',
       'point_coords', 'main_rater_annotation', 'rater1_annotation',
       'geojson_file', 'area', 'diameter', 'size_bracket'],
      dtype='object')

In [37]:
max_output

,main_rater_index,main_rater_class,main_rater_object_name,rater1_index,rater1_class,rater1_object_name,polygon_coords,point_coords,main_rater_annotation,rater1_annotation,geojson_file,area,diameter,size_bracket
0,0,Cored,2:Cored,7,Neuritic,04:Neuritic,"[[[79365, 97527], [79282, 97568], [79282, 9756...","[79445.24, 97878.23]",Polygon(C),PointNone,22650_7_AmyB_1_ceren.mrxs.geojson,250041.0,564.235845,f. >500
1,1,Cored,0:Cored,5,Cored,01:Cored,"[[[76803, 98088], [76723, 98104], [76716, 9810...","[76741.58, 98269.82]",Polygon(C),Point(C),22650_7_AmyB_1_ceren.mrxs.geojson,86083.5,331.066332,e. 250-500
2,2,Coarse-Grained,1:Coarse-Grained,6,Neuritic,03:Neuritic,"[[[77399, 97979], [77396, 97980], [77393, 9798...","[77402.17, 98133.61]",Polygon(CG),PointNone,22650_7_AmyB_1_ceren.mrxs.geojson,85384.0,329.718494,e. 250-500
3,3,Coarse-Grained,3:Coarse-Grained,10,Coarse-Grained,02:Coarse-Grained,"[[[76395, 97151], [76356, 97163], [76349, 9716...","[76407.88, 97316.38]",Polygon(CG),Point(CG),22650_7_AmyB_1_ceren.mrxs.geojson,71358.5,301.424060,e. 250-500
4,4,Coarse-Grained,8:Coarse-Grained,12,Coarse-Grained,22:Coarse-Grained,"[[[72299, 141893], [72293, 141896], [72286, 14...","[72409.2, 142201.91]",Polygon(CG),Point(CG),22650_7_AmyB_1_ceren.mrxs.geojson,322820.5,641.114519,f. >500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,52,Coarse-Grained,5:Coarse-Grained,26,Coarse-Grained,17:Coarse-Grained,"[[[38921, 25973], [38918, 25974], [38917, 2597...","[38954.73, 26068.31]",Polygon(CG),Point(CG),354049_6_AmyB_1_ceren.mrxs.geojson,18929.5,155.247505,d. 100-250
310,53,Cored,52:Cored,36,Cored,10:Cored,"[[[36162, 24922], [36158, 24923], [36155, 2492...","[36188.35, 25003.09]",Polygon(C),Point(C),354049_6_AmyB_1_ceren.mrxs.geojson,17781.5,150.466305,d. 100-250
311,54,Coarse-Grained,0:Coarse-Grained,34,Coarse-Grained,31:Coarse-Grained,"[[[37954, 25100], [37934, 25119], [37929, 2511...","[37949.93, 25162.08]",Polygon(CG),Point(CG),354049_6_AmyB_1_ceren.mrxs.geojson,9869.0,112.096392,d. 100-250
312,48,Diffuse,9:Diffuse,None,None,None,"[[[38777, 28451], [38711, 28478], [38710, 2847...",None,Polygon(D),,354049_6_AmyB_1_ceren.mrxs.geojson,20946.5,163.309253,d. 100-250


In [27]:
max_output.groupby(["main_rater_class","size_bracket"])["main_rater_index"].count().reset_index()

,main_rater_class,size_bracket,main_rater_index
0,Coarse-Grained,c. 50-100,8
1,Coarse-Grained,d. 100-250,57
2,Coarse-Grained,e. 250-500,18
3,Coarse-Grained,f. >500,3
4,Cored,c. 50-100,1
5,Cored,d. 100-250,28
6,Cored,e. 250-500,25
7,Cored,f. >500,1
8,Diffuse,c. 50-100,7
9,Diffuse,d. 100-250,59


In [28]:
compute_kappa_score(max_output, "main_rater_class", "rater1_class")

314


0.27359784801613984

In [29]:
compute_kappa_score(max_output[max_output["rater1_class"]!="None"], "main_rater_class", "rater1_class")

288


0.3432052483598875

In [30]:
max_output[max_output["main_rater_class"]==max_output["rater1_class"]]["main_rater_class"].value_counts()

Diffuse           58
Coarse-Grained    53
Cored             31
Name: main_rater_class, dtype: int64

In [31]:
max_output[max_output["main_rater_class"]!=max_output["rater1_class"]]["main_rater_class"].value_counts()

None              98
Coarse-Grained    33
Cored             24
Diffuse           17
Name: main_rater_class, dtype: int64

In [33]:
max_output.to_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/interrater-study/Kappa_compute/max_harry_objects_full.csv")